Se instala librería transformers

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 16.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.2 MB/s 
     |████████████████████████████████| 120 kB 48.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 25.3 MB/s 
     |████████████████████████████████| 212 kB 59.1 MB/s 
     |████████████████████████████████| 115 kB 70.4 MB/s 
     |████████████████████████████████| 127 kB 67.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Se importan librerías necesarias

In [2]:
import gzip
import shutil
import time
import pandas as pd
import requests
import torch
import torch.nn.functional as F
import torchtext
import transformers
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Se configuran ajustes generales como el dispositivo(CPU o GPU), número de épocas, etc.

In [3]:
torch.backends.cudnn.deterministic = True
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
#Epocas original
#NUM_EPOCHS = 3
NUM_EPOCHS = 1

cuda


Se descarga el Dataset de Reseñas de películas de IMDB y se guarda en un *dataframe*

In [5]:
url = ("https://github.com/rasbt/"
"machine-learning-book/raw/"
"main/ch08/movie_data.csv.gz")
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)
with gzip.open('movie_data.csv.gz', 'rb') as f_in:
    with open('movie_data.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [6]:
df = pd.read_csv('movie_data.csv')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


Se divide el Dataset en conjuntos de entrenamiento,validación, test y se tokenizan.

In [7]:
#Partición original del Dataset
#train_texts = df.iloc[:35000]['review'].values
#train_labels = df.iloc[:35000]['sentiment'].values
#valid_texts = df.iloc[35000:40000]['review'].values
#valid_labels = df.iloc[35000:40000]['sentiment'].values
#test_texts = df.iloc[40000:]['review'].values
#test_labels = df.iloc[40000:]['sentiment'].values

train_texts = df.iloc[:5000]['review'].values
train_labels = df.iloc[:5000]['sentiment'].values
valid_texts = df.iloc[5000:10000]['review'].values
valid_labels = df.iloc[5000:10000]['sentiment'].values
test_texts = df.iloc[10000:15000]['review'].values
test_labels = df.iloc[10000:15000]['sentiment'].values

In [8]:
tokenizer = DistilBertTokenizerFast.from_pretrained(
'distilbert-base-uncased'
)
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
valid_encodings = tokenizer(list(valid_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx])
            for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)
train_dataset = IMDbDataset(train_encodings, train_labels)
valid_dataset = IMDbDataset(valid_encodings, valid_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)
train_loader = torch.utils.data.DataLoader(
train_dataset, batch_size=16, shuffle=True)
valid_loader = torch.utils.data.DataLoader(
valid_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(
test_dataset, batch_size=16, shuffle=False)

Se carga el modelo preentrenado "distilbert-base-uncased"

In [10]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(DEVICE)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=5e-5)
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
)
trainer = Trainer(
model=model,
    args=training_args,
    train_dataset=train_dataset,
    optimizers=(optim, None) # optim and learning rate scheduler
)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

In [11]:
from datasets import load_metric
import numpy as np
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    print("logits",logits)
    # note: logits are a numpy array, not a pytorch tensor
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optim, None)) # optim and learning rate scheduler)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


Se entrena modelo

In [12]:
start_time = time.time()
trainer.train()

***** Running training *****
  Num examples = 5000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 313


Step,Training Loss
10,0.686100
20,0.637600
30,0.468300
40,0.543200
50,0.622400
60,0.364800
70,0.399300
80,0.306700
90,0.312200
100,0.286600




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=313, training_loss=0.3332254065873143, metrics={'train_runtime': 247.6106, 'train_samples_per_second': 20.193, 'train_steps_per_second': 1.264, 'total_flos': 662336993280000.0, 'train_loss': 0.3332254065873143, 'epoch': 1.0})

Se evalúa el modelo

In [13]:
print(trainer.evaluate())

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


logits [[ 1.9214524 -2.119882 ]
 [ 1.8028667 -2.1106424]
 [ 0.8824677 -1.13314  ]
 ...
 [-2.199083   1.9993056]
 [ 1.7626476 -1.9684198]
 [-2.2578118  2.0856004]]
{'eval_loss': 0.2547866106033325, 'eval_accuracy': 0.9008, 'eval_runtime': 93.9676, 'eval_samples_per_second': 53.21, 'eval_steps_per_second': 3.331, 'epoch': 1.0}


Se guarda el modelo

In [14]:
torch.save(model.state_dict(), 'best_model_state.bin')

Se colocan cadenas para validar

In [17]:
review_text  = "bad movie"

encoded_review = tokenizer.encode_plus(
  review_text,
  max_length=200,
  add_special_tokens=True,
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',
)

review_text2 = "I waited long to watch this movie. Also because I like Bruce Willis. The plot was quite different from what I had expected but still quite good. Its a good mix of emotions, humor and drama.<br /><br />Left me thinking over and again :)"
encoded_review2 = tokenizer.encode_plus(
  review_text2,
  max_length=200,
  add_special_tokens=True,
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',
)

In [18]:
class_names = ['negative', 'positive']

input_ids = encoded_review['input_ids'].to(DEVICE)
attention_mask = encoded_review['attention_mask'].to(DEVICE)
output = model(input_ids, attention_mask)


input_ids2 = encoded_review2['input_ids'].to(DEVICE)
attention_mask2 = encoded_review2['attention_mask'].to(DEVICE)
output2 = model(input_ids2, attention_mask2)


#print(output)
_, prediction = torch.max(output['logits'], dim=1)
_, prediction2 = torch.max(output2['logits'], dim=1)
#print(prediction)
print(f'Review text: {review_text}')
print(f'Sentiment  : {class_names[prediction]}')

print(f'Review text: {review_text2}')
print(f'Sentiment  : {class_names[prediction2]}')

Review text: bad movie
Sentiment  : negative
Review text: I waited long to watch this movie. Also because I like Bruce Willis. The plot was quite different from what I had expected but still quite good. Its a good mix of emotions, humor and drama.<br /><br />Left me thinking over and again :)
Sentiment  : positive
